# wav2vec-base 모델을 가져와 한국인 영어 발화 데이터로 파인튜닝하는 코드입니다.
***
## **토큰화 간 패키지 충돌 방지**
모델 저장할때 충돌이 또 일어나기 때문에, 모델은 save 후 직접 repo에 탑재하였습니다...

In [ ]:
!pip install transformers==4.24.0
!pip uninstall huggingface_hub # Y 입력하세요
!pip install huggingface_hub==0.13.2
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.9 MB/s eta 0:00:00
Found existing installation: huggingface-hub 0.15.1
Uninstalling huggingface-hub-0.15.1:
  Would remove:
    /usr/local/bin/huggingface-cli
    /usr/local/lib/python3.10/dist-packages/huggingface_hub-0.15.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/huggingface_hub/*
Proceed (Y/n)? Y
  Successfully uninstalled huggingface-hub-0.15.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 14.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

UsageError: Line magic function `%%capture` not found.


In [ ]:
# 음성인식 metric 패키지
%%capture
!pip install jiwer

In [ ]:
# 모델의 체크포인트를 저장하기 위해 git LFS를 설치 (왜 저장해야되는지는 모르겠음..)
%%capture
!apt install git-lfs

In [ ]:
#!pip uninstall datasets
#!pip install datasets==2.0.0
#!pip uninstall tokenizers
#!pip install tokenizers==0.13.2
#!pip uninstall transformers
#!pip install transformers==4.18.0
#!pip uninstall huggingface_hub
#!pip install huggingface_hub==0.10.0
#!pip install huggingface_hub

import huggingface_hub
import transformers
import tokenizers
print(huggingface_hub.__version__, transformers.__version__, tokenizers.__version__)

0.13.2 4.24.0 0.13.3


# **Fine-tuning Wav2Vec2 🤗**
추후 cuda 연산을 진행하기 때문에 GPU를 할당해야 됩니다. 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun  7 06:05:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

학습된 모델을 바로 불러올 수 있도록 [🤗 Hub](https://huggingface.co/) 에 저장

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Prepare Data, Tokenizer, Feature Extractor

In [ ]:
from datasets import load_dataset, load_metric
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re
import torch

In [ ]:
# 사전에 저장해둔 데이터를 불러옵니다
data = load_dataset('pongjin/en_with_korean') 

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/pongjin___parquet/pongjin--en_with_korean-759382c4219c4f3e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# train test 9:1로 분할합니다
data = data['train'].train_test_split(test_size=0.1)
# 형식 확인
data

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 500
    })
})

In [ ]:
# 랜덤한 target을 보여주는 함수입니다.

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset." # 샘플이 데이터보다 크면 에러 발생
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html())) 

In [ ]:
show_random_elements(data['train'].remove_columns(["audio"]), num_examples=10)

,text
0,i think you've made a mistake in your calculation
1,very good it will be fifty dollars a person
2,not so i mentioned that you can't see it without sitting up leaning over
3,i've heard that from a lot of people i should check it out
4,that is important information thank you
5,i'm killing myself to support my family
6,i don't know why i'm so hungry right now
7,i was never going to look at them never
8,my name is written in english on the lower part
9,it seems fine how much is the quantity


In [ ]:
# 텍스트 내 특수문자 제거, 소문자 변경 > 특수문자는 소리 단위가 아니기에 인식을 더 어렵게 할 수 있음 또한 언어모델이 없기에 좀 더 robust한 인식이 필요함
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
data = data.map(remove_special_characters)

# 예시 출력
show_random_elements(data['train'].remove_columns(["audio"]))

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

,text
0,i may have been singing that song in my head haha
1,dad it seems so hard to make friends in high school
2,i'll give you a massage on your neck and shoulders
3,yeah that's exactly what i'm talking about
4,i didn't know that i will check them right away
5,which room number should i bring it to
6,go straight ahead and you'll walk into it
7,i've booked a seat on flight two six one to los angeles on friday morning
8,no i'm afraid i'm not in austin still in kansas city
9,do you have any automatic ticket office


In [ ]:
# 문자열 기준 임베딩을 하기 위해 배치 내 모든 문자를 추출하는 함수
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=False, remove_columns=data.column_names['train'])

vocabs

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
})

In [ ]:
# train test에서 나오는 모든 문자 반환
vocab_list = list(set(vocabs['train']["vocab"][0]) | set(vocabs['test']["vocab"][0]))

In [ ]:
# vocab 구축
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{"'": 0,
 'u': 1,
 'y': 2,
 'z': 3,
 'c': 4,
 'r': 5,
 'n': 6,
 ' ': 7,
 'q': 8,
 'k': 9,
 'v': 10,
 's': 11,
 'h': 12,
 'e': 13,
 'd': 14,
 'b': 15,
 'p': 16,
 'o': 17,
 'l': 18,
 'g': 19,
 'i': 20,
 'f': 21,
 'j': 22,
 'w': 23,
 'a': 24,
 'm': 25,
 't': 26,
 'x': 27}

In [ ]:
# 띄어쓰기를 | 로 바꾸어 표현
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
# oov를 위해 unk 추가 및 패딩 토큰 추가
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [ ]:
# save the vocabulary as a json
import json

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
# In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

추후 토크나이저 재사용을 위해 [🤗 Hub](https://huggingface.co/) 에 업로드

In [ ]:
#from huggingface_hub import create_repo
repo_name = "pongjin/en_with_korean_tokenizer_960h"
#create_repo(repo_name, private=True, repo_type='model')

and upload the tokenizer to the [🤗 Hub](https://huggingface.co/).

In [ ]:
tokenizer.push_to_hub(repo_name, token="토큰")

CommitInfo(commit_url='https://huggingface.co/pongjin/en_with_korean_tokenizer_960h/commit/c13aa7fd39e408cf4b02ce81a2230cfe3fcda6aa', commit_message='Upload tokenizer', commit_description='', oid='c13aa7fd39e408cf4b02ce81a2230cfe3fcda6aa', pr_url=None, pr_revision=None, pr_num=None)

### Create Wav2Vec2 Feature Extractor
원 음성 데이터를 샘플링하여 컴퓨터가 인식할 수 있는 이산화 처리를 하기 위한 특성 추출 과정  
1. feature_size : 입력 차원으로 입력은 1차원 음성이기에 1로 설정
2. sampling_rate : wav2vec 는 16khz로 pre-trained 되었기에 16000으로 설정.  
(만약  음성 데이터가 다른 khz로 되어있다면 변환하는 resampling 과정이 필요)  
3. padding_value : 패딩할 값 지정
4. do_normalize : 기본적으로 정규화 하는것이 성능이 좋음
5. return_attention_mask : 모델이 학습할 때 attention_mask를 사용할지 여부. 일반적으로 모델은 패딩된 토큰을 마스크하기 위해 항상 Attention_mask를 사용해야 하지만, Wav2Vec2_base는 매우 구체적인 디자인으로 인해 Attention_mask를 사용하지 않을 때 더 나은 결과를 얻는다고 합니다. 다른 음성 모델에는 권장하지 않습니다. 이 노트북을 사용하여 large-lv60를 튜닝하려면 이 매개변수를 True로 설정해야 합니다. (왜 그런지는 모르겠음..)



In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

Processor는 feature extractor와 tokenizer를 한번에 처리해주는 파이프라인으로 예측 간 model과 processor만 있으면 됨

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Preprocess Data

In [ ]:
data['train'][0]["audio"]

{'path': 'SW201105ETRNGSM06044JBY0081.wav',
 'array': array([0.00000000e+00, 3.05175781e-05, 0.00000000e+00, ...,
        3.05175781e-05, 3.05175781e-05, 0.00000000e+00]),
 'sampling_rate': 16000}

재생해보기

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(data))

print(data['train'][rand_int]["text"])
ipd.Audio(data=np.asarray(data['train'][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

i thought i would have fun but it's not 


In [ ]:
# processor 적용 함수
def prepare_dataset(batch):
    audio = batch["audio"]

    # 인풋 전처리
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"]) #길이는 왜 가져가는거쥬
    
    with processor.as_target_processor(): # 텍스트 레이블을 모델이 이해할 수 있는 형식으로 변환해줌..?
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
data = data.map(prepare_dataset, remove_columns=data.column_names['train'], num_proc=2) # 2개로 병렬처리

Map (num_proc=2):   0%|          | 0/4500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


## Training & Evaluation

 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) 참고

- Data collator를 정의합니다. 
대부분의 NLP 모델과 달리 Wav2Vec2는 출력 길이보다 입력 길이가 훨씬 더 큽니다. 예: 입력 길이가 50000인 샘플의 출력 길이는 100 이하입니다. 큰 입력 크기가 주어지면 훈련 배치를 동적으로 채우는 것이 훨씬 더 효율적입니다. 즉, 모든 훈련 샘플은 전체 가장 긴 샘플이 아니라 배치에서 가장 긴 샘플에만 채워져야 합니다. 따라서 Wav2Vec2를 미세 조정하려면 아래에서 정의할 특별한 padding data collator가 필요합니다.

- Evaluation metric 정의. 
훈련하는 동안 모델은 단어 오류율로 평가되어야 합니다. 그에 따라 compute_metrics 함수를 정의해야 합니다.
사전 훈련된 체크포인트를 로드합니다. 사전 훈련된 체크포인트를 로드하고 훈련을 위해 올바르게 구성해야 합니다.
- training configuration을 정의합니다.
모델을 미세 조정한 후 테스트 데이터에서 모델을 올바르게 평가하고 실제로 음성을 올바르게 전사하는 방법을 학습했는지 확인합니다.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

너무 많은 세부 사항에 들어가지 않고 일반적인 데이터 수집기와 달리 이 데이터 수집기는 input_values와 labels를 다르게 처리하므로 별도의 패딩 기능에 적용됩니다(다시 Wav2Vec2의context manager 사용). 이것은 음성에서 입력과 출력이 동일한 패딩 기능으로 처리되어서는 안 되는 서로 다른 양식이기 때문에 필요합니다. 일반적인 데이터 수집기와 유사하게 레이블의 패딩 토큰은 '-100'이므로 손실을 계산할 때 해당 토큰이 고려되지 않습니다.

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding: 
    """
    패딩 길이를 정하는 방법
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-33-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions # 예측결과는 sequence of logits로 argmax를 통해 가장 확률이 높은 문자열 반환 
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id # 패딩토큰변경

    pred_str = processor.batch_decode(pred_ids) # 디코딩(같은 문자 결합 + 마스크 제거 등)
   
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False) # 디코딩(인코딩된 레이블을 단어로 변환) 과정에서 같은 문자가 합쳐지는 과정이 진행되지 않게 해야됨

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

 CTC에서는 동일한 연속 토큰을 단일 토큰으로 그룹화합니다. 그러나 인코딩된 레이블은 모델의 예측 토큰과 일치하지 않으므로 디코딩할 때 그룹화하지 않아야 합니다. 이것이 group_tokens=False 매개변수를 전달해야 하는 이유입니다. 이 매개변수를 전달하지 않으면 "hello"와 같은 단어가 잘못 인코딩되고 "helo"로 디코딩됩니다.

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h",
    ctc_loss_reduction="mean", #GPU 메모리 절약
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 매개변수 참고(출처 GPT)
### **ctc_loss_reduction**  
CTC (Connectionist Temporal Classification) 손실 함수에서 사용되는 감소(reduction) 옵션을 나타냅니다. CTC는 음성 인식과 같은 시퀀스 레이블링 작업에 많이 사용되는 손실 함수입니다.

CTC 손실은 입력 시퀀스와 해당 시퀀스의 레이블 사이의 차이를 계산하여 모델을 학습시킵니다. 이때, CTC 손실의 계산에는 감소 옵션이 필요한데, 이는 손실 값의 감소 방식을 지정하는 매개변수입니다.

ctc_loss_reduction의 가능한 값은 주로 다음과 같이 정의됩니다:

'none': 손실 값을 감소시키지 않고, 모든 시퀀스 레이블 간의 손실을 독립적으로 계산합니다.
'sum': 시퀀스 내 모든 타임 스텝에 대한 손실 값을 합산합니다.
'mean': 시퀀스 내 모든 타임 스텝에 대한 손실 값을 평균합니다.
'batchmean': 배치 내 모든 시퀀스의 손실 값을 평균합니다.
각각의 옵션은 CTC 손실 함수의 결과를 다르게 처리합니다. 예를 들어, 'mean' 옵션은 시퀀스 내 모든 타임 스텝의 손실 값을 평균하여 하나의 스칼라 손실 값으로 반환합니다.

CTC 손실 함수를 사용하는 프레임워크나 라이브러리에 따라 ctc_loss_reduction 옵션의 지원과 사용법이 다를 수 있으므로, 해당 프레임워크의 문서를 참조하는 것이 좋습니다.
***
### **Gradient Checkpointing**  
신경망 모델에서 메모리 사용량을 줄이고 계산 속도를 향상시키기 위한 기술입니다. 특히, 매우 깊거나 매개변수가 많은 모델에서 유용합니다.

일반적으로 역전파 알고리즘은 순전파 과정에서 계산된 모든 중간값을 메모리에 보관하여 그래디언트 계산에 사용합니다. 하지만 모델이 매우 큰 경우, 중간값을 모두 저장하는 것은 메모리 요구량이 많아질 수 있습니다.

Gradient Checkpointing은 중간값 중 일부만 저장하는 방법을 제공합니다. 이를 위해 중간 연산 결과를 메모리에 캐시하고 필요할 때만 계산하는 방식을 사용합니다. 순전파 과정 중에 중간 연산 결과를 캐시하고, 역전파에서는 필요한 그래디언트만을 계산합니다.

이러한 방식으로 Gradient Checkpointing은 메모리 사용량을 줄이고, 필요한 그래디언트만 계산하므로 계산 속도를 향상시킬 수 있습니다. 하지만 일부 연산을 다시 계산해야하는 오버헤드가 발생할 수 있으므로, 메모리와 계산 속도 사이의 트레이드오프 관계가 있습니다.

Gradient Checkpointing은 주로 대규모 모델, 특히 비디오 분류, 세그멘테이션, 자연어 처리 등의 작업에 사용됩니다. 프레임워크 및 라이브러리에 따라 Gradient Checkpointing을 구현하고 사용하는 방법이 다를 수 있으므로, 해당 프레임워크의 문서를 참조하는 것이 좋습니다.

In [ ]:
# 모델 구조
print(model)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [ ]:
# 논문에서 이미 충분히 훈련되었다고 하기에 모델의 특징 추출하는 CNN 부분은 학습을 하지 않는다. 학습하면 오히려 성능이 더 떨어질수도..?
model.freeze_feature_encoder()

In [ ]:
# 하이퍼 파라미터 설정
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True, #입력 길이가 유사한 샘플을 하나의 배치로 그룹화. 쓸모없는 패딩 토큰의 전체 수를 크게 줄여 보다 학습 시간을 단축(동적 패딩이라 가능한듯)
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=15,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  # 아래 두개는 timit data에서 경험적으로 설정한 것으로 다른 데이터셋은 직접 조정해볼 필요가 있음
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
# 파이프라인 구축
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


### Training

긴 학습 도중 코랩이 자동으로 중지되는것을 막기 위해 F12 누르고 아래 콘솔창에 해당 코드 넣으면 자동으로 클릭해줌.

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.train() #배치 16 / 에폭 15 / GPU A100 / 모델 w2v_base_960h

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4500
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4230
  Number of trainable parameters = 90195872
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and wil

Step,Training Loss,Validation Loss,Wer
500,9.633500,2.133904,0.989419
1000,1.356100,0.510951,0.363986
1500,0.771500,0.419496,0.275875
2000,0.455000,0.353964,0.239515
2500,0.432800,0.352950,0.235090
3000,0.260700,0.338241,0.227780
3500,0.220300,0.337511,0.220277
4000,0.220600,0.336569,0.217968


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to pongjin/en_with_korean_tokenizer_960h/checkpoint-500
Configuration saved in pongjin/en_with_korean_tokenizer_960h/checkpoint-500/config.json
Model weights saved in pongjin/en_with_korean_tokenizer_960h/checkpoint-500/pytorch_model.bin
Feature extractor saved in pongjin/en_with_korean_tokenizer_960h/checkpoint-500/preprocessor_config.json
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio in

TrainOutput(global_step=4230, training_loss=1.5873025934746925, metrics={'train_runtime': 4914.0948, 'train_samples_per_second': 13.736, 'train_steps_per_second': 0.861, 'total_flos': 5.112723943644537e+18, 'train_loss': 1.5873025934746925, 'epoch': 15.0})

In [ ]:
# push to hub는 에러가 뜨기에 직접 코랩 로컬에 저장 후 다운로드
trainer.save_model("/content/sample_data")

Saving model checkpoint to /content/sample_data
Configuration saved in /content/sample_data/config.json
Model weights saved in /content/sample_data/pytorch_model.bin
Feature extractor saved in /content/sample_data/preprocessor_config.json


### Evaluate

Let's load the `processor` and `model`.

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

processor2 = Wav2Vec2Processor.from_pretrained("pongjin/en_with_korean_w2v_model_960h")
model2 = Wav2Vec2ForCTC.from_pretrained("pongjin/en_with_korean_w2v_model_960h").cuda() # 쿠다는 gpu 연산용

loading configuration file preprocessor_config.json from cache at /root/.cache/huggingface/hub/models--pongjin--en_with_korean_w2v_model_960h/snapshots/889cc1d3da676546ae6d9d05400ac6e3fa4fad3b/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}



loading file vocab.json from cache at /root/.cache/huggingface/hub/models--pongjin--en_with_korean_w2v_model_960h/snapshots/889cc1d3da676546ae6d9d05400ac6e3fa4fad3b/vocab.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--pongjin--en_with_korean_w2v_model_960h/snapshots/889cc1d3da676546ae6d9d05400ac6e3fa4fad3b/tokenizer_config.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--pongjin--en_with_korean_w2v_model_960h/snapshots/889cc1d3da676546ae6d9d05400ac6e3fa4fad3b/special_tokens_map.json
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--pongjin--en_with_korean_w2v_model_960h/snapshots/889cc1d3da676546ae6d9d05400ac6e3fa4fad3b/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base-960h",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--pongjin--en_with_korean_w2v_model_960h/snapshots/889cc1d3da676546ae6d9d05400ac6e3fa4fad3b/pytorch_model.bin
All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

All the weights of Wav2Vec2ForCTC were initialized from the model checkpoint at pongjin/en_with_korean_w2v_model_960h.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Wav2Vec2ForCTC for predictions without further training.


map을 통해 예측 후 텍스트와 전사를 가져온다

**주의**: [issue](https://github.com/pytorch/fairseq/issues/3227)과 같은 오류로 인해 `batch_size=1`로 진행한다.  
패딩된 입력은 패딩 되지 않은 입력과 정확히 동일한 출력을 생성하지 않으므로 입력을 전혀 패딩 하지 않음으로써 더 나은 WER을 얻을 수 있다고 한다.(이게 무슨 의미...?)

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model2(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor2.batch_decode(pred_ids)[0]
  batch["text"] = processor2.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = data["test"].map(map_to_result, remove_columns=data["test"].column_names)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Let's compute the overall WER now.

In [ ]:
# 저장된 모델
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.121


In [ ]:
show_random_elements(results)

,pred_str,text
0,hope you think better of the metter,hope you'll think better of the matter
1,ok youw'ill get twenty percents off the least price,ok you will get twenty percent off the list price
2,used te talk about a particular man in a negative or inselrting way,used to talk about a particular man in a negative or insulting way
3,do you have the same as like this but bigger,do you have the same as like this but bigger
4,i can't wait for that ou vidio for lundov lun,i can't wait for their new video for run devil run
5,thank you for the kind mention blessings,thank you for the kind mention blessings
6,do you have food that is not too spice for a child,do you have food that is not too spicy for a child
7,i'd like to see some dancing do you have any information,i'd like to see some dancing do you have any information
8,i took a spanish class then i went to japan for a week,i took a spanish class then i went to japan for a week
9,would you mind if i called on you to ask for your advice mr carneggi,would you mind if i called on you to ask for your advice mr carnegie


예측된 전사가 정답 전사와 음향적으로 매우 유사하지만 종종 철자나 문법 오류가 포함되어 있음.

언어 모델을 사용하지 않고 순수하게 Wav2Vec2에 의존한다는 점을 감안할 때 적절한 결과.

CTC가 실제로 어떻게 작동하는지 출력을 보여주는 코드

In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(data["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


"[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] i t t ' ' s s [PAD] | | b b e [PAD] e e n [PAD] [PAD] [PAD] [PAD] | t t [PAD] [PAD] [PAD] [PAD] [PAD] o u f f [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | b [PAD] u u t t | [PAD] [PAD] i [PAD] [PAD] | c [PAD] [PAD] a n n [PAD] [PA